# Carga de librerías y métodos

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import torch
from torch.serialization import safe_globals
from PIL import Image
from IPython.display import display



In [ ]:
ckpt_path = "/home/joan_ds/Sandbox/UOC/TFM/tfm_deeplab/DeepLabV3Plus-Pytorch/best_deeplabv3plus_mobilenet_cityscapes_os16.pth"
print("¿Existe?", os.path.isfile(ckpt_path), ckpt_path)


try:
    ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)
except Exception:
    with safe_globals([np.core.multiarray.scalar]):
        ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=True)

print(type(ckpt), ckpt.keys() if isinstance(ckpt, dict) else "no dict")

In [ ]:
input_dir = '/home/joan_ds/Sandbox/UOC/TFM/trials_imgs'
tmp_dir = 'DeepLabV3Plus-Pytorch/tmp_resized'
output_dir = 'DeepLabV3Plus-Pytorch/output'

In [ ]:
CITYSCAPES_ID2COLOR = {
    0: (128, 64, 128),  # road
    1: (244, 35, 232)   # sidewalk
}

def filter_mask_ids(ids_map):
    h, w = ids_map.shape
    mask_rgb = np.zeros((h, w, 3), dtype=np.uint8)
    present_classes = []
    for class_id, color in CITYSCAPES_ID2COLOR.items():
        m = (ids_map == class_id)
        if m.any():
            mask_rgb[m] = color
            present_classes.append(class_id)
    return mask_rgb, present_classes

def mask_pred(resample_method: dict, input_dir=input_dir, tmp_dir=tmp_dir, output_dir=output_dir):
    method_name  = resample_method['name']
    method_Image = resample_method['method_Image']
    method_cv2   = resample_method['method_cv2']

    results_lst = []
    

    os.makedirs(tmp_dir, exist_ok=True)

    for fname in os.listdir(input_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(input_dir, fname)
            img = Image.open(img_path).convert("RGB")
            img.resize((2048, 1024), method_Image).save(
                os.path.join(tmp_dir, os.path.splitext(fname)[0] + '.png')
            )
            

    !cd DeepLabV3Plus-Pytorch && \
    python /home/joan_ds/Sandbox/UOC/TFM/tfm_deeplab/DeepLabV3Plus-Pytorch/predict.py \
        --model deeplabv3plus_mobilenet \
        --dataset cityscapes \
        --ckpt /home/joan_ds/Sandbox/UOC/TFM/tfm_deeplab/DeepLabV3Plus-Pytorch/best_deeplabv3plus_mobilenet_cityscapes_os16.pth \
        --input "tmp_resized" \
        --save_val_results_to output

    for fname in os.listdir(output_dir):
        if not fname.endswith('_ids.png'):
            continue

        name = fname.replace('_ids.png', '')
        dicc = {'method_name': method_name, 'image_name': fname}
        ids_path  = os.path.join(output_dir, fname)
        conf_path = os.path.join(output_dir, name + '_conf.png')

        # trobar original
        orig_path = next(
            (os.path.join(input_dir, name + ext) for ext in ('.jpg', '.jpeg', '.png')
             if os.path.exists(os.path.join(input_dir, name + ext))),
            None
        )
        if orig_path is None: 
            continue

        original = cv2.cvtColor(cv2.imread(orig_path), cv2.COLOR_BGR2RGB)
        ids_map  = cv2.imread(ids_path,  cv2.IMREAD_GRAYSCALE)
        conf_map = cv2.imread(conf_path, cv2.IMREAD_GRAYSCALE) if os.path.exists(conf_path) else None

        
        road_pct     = float((ids_map == 0).mean())
        sidewalk_pct = float((ids_map == 1).mean())
        n_classes = len(np.unique(ids_map))
        dicc |= {'n_classes': n_classes, 'road_pct': round(road_pct * 100, 3), 'sidewalk_pct': round(sidewalk_pct * 100, 3)}
        results_lst.append(dicc)

        # confiances (si tenim conf_map)
        road_conf = sidewalk_conf = np.nan
        if conf_map is not None:
            conf_float = conf_map.astype(np.float32) / 255.0
            m0 = (ids_map == 0); m1 = (ids_map == 1)
            if m0.any(): road_conf     = float(conf_float[m0].mean())
            if m1.any(): sidewalk_conf = float(conf_float[m1].mean())
            dicc |= {'road_conf': road_conf, 'sidewalk_conf': sidewalk_conf}

        print(f"\nEn {name} se detectan {n_classes} clases diferentes.")
        print(f"Proporciones sobre píxels:\nroad: {road_pct * 100:.3f} %\nsidewalk: {sidewalk_pct * 100:.3f} %",
              f"\nConfianza media en la predicción (logits):\nmeanConf(road) = {road_conf:.3f}\nmeanConf(sidewalk) = {sidewalk_conf:.3f}")

        # màscara RGB i overlay
        mask_rgb, present_classes = filter_mask_ids(ids_map)
        mask_resized = cv2.resize(mask_rgb, (original.shape[1], original.shape[0]),
                                  interpolation=method_cv2)
        overlay = cv2.addWeighted(original, 0.5, mask_resized, 0.5, 0)

        # llegenda
        legend_patches = []
        for class_id in present_classes:
            color_rgb = tuple(np.array(CITYSCAPES_ID2COLOR[class_id]) / 255.0)
            label = "road" if class_id == 0 else "sidewalk"
            legend_patches.append(plt.Rectangle((0,0),1,1, fc=color_rgb, label=label))

        # títol amb resum “proxy”
        present_n = len(present_classes)
        if np.isnan(road_conf):     road_conf_txt = "—"
        else:                       road_conf_txt = f"{road_conf:.3f}"
        if np.isnan(sidewalk_conf): sidewalk_conf_txt = "—"
        else:                       sidewalk_conf_txt = f"{sidewalk_conf:.3f}"

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(original); axes[0].set_title(f'Original\nImagen {name}'); axes[0].axis('off')
        axes[1].imshow(overlay)
        axes[1].set_title(
            f"{method_name} – {present_n} clases presentes de las 2 clases de interés\n"
            f"road: {road_pct * 100:.3f} %, conf {road_conf_txt} | "
            f"sidewalk: {sidewalk_pct * 100:.3f} %, conf {sidewalk_conf_txt}"
        )
        axes[1].axis('off')
        if legend_patches:
            fig.legend(handles=legend_patches, loc='lower center', ncol=len(legend_patches))
        plt.tight_layout(); plt.show()
    
    return results_lst

        

# Experimentos con imágenes del dataset

In [ ]:
linear = {'name': 'LINEAR', 'method_Image': Image.BILINEAR, 'method_cv2': cv2.INTER_LINEAR}
nearest = {'name': 'NEAREST', 'method_Image': Image.NEAREST, 'method_cv2': cv2.INTER_NEAREST}
cubic = {'name': 'CUBIC', 'method_Image': Image.BICUBIC, 'method_cv2': cv2.INTER_CUBIC}

methods_lst = [linear, nearest, cubic]

In [ ]:
results_dataset = []

for method in methods_lst:
    sup_title = f"Resultados tras redimensionar las imaǵenes con el método de remuestreo {method['name']}"

    print(sup_title)

    results_dataset.append(mask_pred(method))


# Experimentos con imágenes de los benchmarks

In [ ]:
bdd100k = {'name': 'BDD100K', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/BDD100K'}
cityscapes = {'name': 'Cityscapes', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/Cityscapes'}
mapillary_vistas = {'name': 'Mapillary Vistas', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/Mapillary_Vistas'}

benchmark_lst = [bdd100k, cityscapes, mapillary_vistas]


In [ ]:
results_benchmarks_lst = []

for benchmark in benchmark_lst:
    print (f"Segmentación con el submuestreo del dataset {benchmark['name']}.")

    for method in methods_lst:
        sup_title = f"Resultados tras redimensionar las imaǵenes con el método de remuestreo {method['name']}"

        print(sup_title)

        results_benchmark = mask_pred(method, input_dir=benchmark['path'])
        results_benchmarks_lst.append(results_benchmark)


# Output

In [ ]:
results_benchmarks_flatten = [dicc for lst in results_benchmarks_lst for dicc in lst]

results_dataset_flatten = [dicc for lst in results_dataset for dicc in lst]

results_dataset_df = pd.DataFrame(results_dataset_flatten)
results_benchmarks_df = pd.DataFrame(results_benchmarks_flatten)

df = pd.concat([results_dataset_df, results_benchmarks_df]).reset_index(drop=True)

df.info()

In [ ]:
df[df.image_name == df.image_name[0]]

In [ ]:
df.to_excel('20250813_results_analysis.xlsx')